$ln(z) =\frac{z-1}{1+} \frac{1^2(z-1)}{2+} \frac{1^2(z - 1)}{3+} \frac{2^2(z-1)}{4+} \frac{2^2(z-1)}{5+} \frac{3^2(z-1)}{6+} \frac{3^2(z-1)}{7+}$

$= \frac{z - 1}{1 + \frac{1^2(z-1)}{2 + \frac{1^2(z-1)}{3+\dots}}}$

$b_0(z) + \frac{a_1(z)}{b_1(z)+\frac{a_2(z)}{b_2(z) + ...}}$

In [35]:
def ln(z):
    b0 = 0
    a = 1
    b = 1

    Ajm2 = 1
    Bjm2 = 0
    Ajm1 = b0
    Bjm1 = 1

    ynm1 = 0
    tol = 1e-9

    for j in range(1, 501):
        new_term = a * a * (z - 1)
        Aj = b*Ajm1 + new_term * Ajm2
        Bj = b*Bjm1 + new_term * Bjm2
        yn = Aj/Bj
        diff = yn - ynm1

        if abs(diff) < tol:
            return yn
        ynm1 = yn
        Ajm2 = Ajm1
        Bjm2 = Bjm1
        Ajm1 = Aj
        Bjm1 = Bj
        # Increment after 3 terms after each term has been used twice
        if j >= 3 and j % 2:
            a += 1
        b += 1

In [34]:
from random import random

import math
n_tests = 100
tol = 1e-8

failed = False
for _ in range(n_tests):
    check = random() * 15
    res = ln(check)
    z_roundtrip = math.exp(res)
    delta = abs(z_roundtrip - check)
    if delta > tol:
        print(f"\n----------- FAILED ----------- {_}/{n_tests} Error: ±", delta, "\n")
        failed = True

if not failed:
    print(f"ALL {n_tests} TESTS PASSED")


ALL 100 TESTS PASSED


For fun, I also implemented a more modular continued fraction expansion that works on any other problem that uses continued fractions beyond just ln.

That implementation is below, with its own test suite.

In [32]:
import itertools

def CF(a, b, b0):
    Ajm2 = 1
    Bjm2 = 0
    Ajm1 = b0
    Bjm1 = 1

    ynm1 = 0
    tol = 1e-9
    i = 1

    for a_term, b_term in zip(a, b):
        if i > 1000:
            print("FAILED TO CONVERGE")
            break
        Aj = b_term * Ajm1 + a_term * Ajm2
        Bj = b_term * Bjm1 + a_term * Bjm2
        yn = Aj/Bj
        diff = yn - ynm1

        # print(i, a_term, b_term, Aj, Bj, yn)
        i += 1

        if abs(diff) < tol:
            return yn

        ynm1 = yn
        Ajm2 = Ajm1
        Bjm2 = Bjm1
        Ajm1 = Aj
        Bjm1 = Bj

def functional_ln(x):
    def num(term):
        yield term - 1
        a_term = 1
        used_twice = False
        while True:
            yield a_term * a_term * (term - 1)
            if used_twice:
                a_term += 1
            used_twice = not used_twice

    def den(term):
        for x in itertools.count(1):
            yield x

    return CF(num(x), den(x), 0)

In [39]:
import math
n_tests = 100
tol = 1e-8

failed = False
for _ in range(n_tests):
    check = random() * 15
    res = functional_ln(check)
    z_roundtrip = math.exp(res)
    delta = abs(z_roundtrip - check)
    if delta > tol:
        print(f"\n----------- FAILED ----------- {_}/{n_tests} Error: ±", delta, "\n")
        failed = True

if not failed:
    print(f"ALL {n_tests} TESTS PASSED")


ALL 100 TESTS PASSED
